In [1]:
import argparse
import os
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from torchvision.models import *

from triplet_whale_loader import *
from tripletnet import Tripletnet
from configure import *
from losses import *
from embedding import *
config = Config()

Using GPU


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.base_model = resnet50(pretrained=True)
        self.base_model.fc = nn.Linear(in_features=2048, out_features=128, bias=True)
        # print(self.base_model)

    def forward(self, x):
        # print("X_shape {}".format(x.shape))
        return self.base_model(x)

In [3]:
embedding = EmbeddingL2()
tnet = Tripletnet(embedding)
if config.USE_GPU:
    tnet.cuda()

In [4]:
# optionally resume from a checkpoint
resume = "./runs/WhaleTriplet/checkpoint.pth.tar"
if os.path.isfile(resume):
    print("=> loading checkpoint '{}'".format(resume))
    checkpoint = torch.load(resume)
    # args.start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    tnet.load_state_dict(checkpoint['state_dict'])
    print("=> loaded checkpoint '{}' (epoch {})"
            .format(resume, checkpoint['epoch']))
else:
    print("=> no checkpoint found at '{}'".format(resume))

=> loading checkpoint './runs/WhaleTriplet/checkpoint.pth.tar'
=> loaded checkpoint './runs/WhaleTriplet/checkpoint.pth.tar' (epoch 3)


In [5]:
import glob
train_files = glob.glob("../../input/train/*.jpg")
test_files = glob.glob("../../input/test/*.jpg")

In [6]:
test_transform = transforms.Compose([
    transforms.Resize(config.SZ),
    transforms.ToTensor(),
])

def load_image(path):
    img = default_image_loader(path)
    img = test_transform(img)
    img = img.numpy()
    return img

In [7]:
def data_generator(fpaths, batch=16):
    i = 0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        img = load_image(path)
        imgs.append(img)
        fnames.append(os.path.basename(path))
        if i == batch:
            i = 0
            imgs = np.array(imgs)
            yield fnames, imgs

    if i < batch:
        imgs = np.array(imgs)
        yield fnames, imgs

    raise StopIteration()

In [8]:
train_preds = []
train_file_names = []
i = 1
for fnames, imgs in tqdm(data_generator(train_files, batch=32), total=len(train_files) // 32):
    i += 1
    imgs = torch.from_numpy(imgs)
    if config.USE_GPU:
        imgs = imgs.cuda()
    imgs = Variable(imgs)
    predicts = tnet.get_embedded(imgs)
    predicts = predicts.cpu().data.numpy().tolist()
    train_preds += predicts
    train_file_names += fnames

train_preds = np.array(train_preds)

308it [01:26,  3.58it/s]                         


In [9]:
test_preds = []
test_file_names = []
for fnames, imgs in tqdm(data_generator(test_files, batch=32), total=len(test_files) // 32):
    imgs = torch.from_numpy(imgs)
    if config.USE_GPU:
        imgs = imgs.cuda()
    imgs = Variable(imgs)
    predicts = tnet.get_embedded(imgs)
    predicts = predicts.cpu().data.numpy().tolist()
    test_preds += predicts
    test_file_names += fnames

test_preds = np.array(test_preds)

488it [02:13,  3.67it/s]                         


In [11]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=6)
neigh.fit(train_preds)

distances_test, neighbors_test = neigh.kneighbors(test_preds)
distances_test, neighbors_test = distances_test.tolist(), neighbors_test.tolist()

In [12]:
preds_str = []

file_id_mapping = {k: v for k, v in zip(config.TRAIN_DF.Image.values, config.TRAIN_DF.Id.values)}

for filepath, distance, neighbour_ in zip(test_file_names, distances_test, neighbors_test):
    sample_result = []
    sample_classes = []
    for d, n in zip(distance, neighbour_):
        train_file = train_files[n].split(os.sep)[-1]
        class_train = file_id_mapping[train_file]
        sample_classes.append(class_train)
        sample_result.append((class_train, d))

    if "new_whale" not in sample_classes:
        sample_result.append(("new_whale", 0.1))
    sample_result.sort(key=lambda x: x[1])
    sample_result = sample_result[:5]
    preds_str.append(" ".join([x[0] for x in sample_result]))

df = pd.DataFrame(preds_str, columns=["Id"])
df['Image'] = [x.split(os.sep)[-1] for x in test_file_names]

In [13]:
df.to_csv('triplet_resnet50_new_model.csv', index=False)